# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 3 2022 12:55PM,247755,19,ACG-2102491445,ACG North America LLC,Released
1,Oct 3 2022 12:30PM,247719,20,ALU0008046,Alumier Labs Inc.,Released
2,Oct 3 2022 12:29PM,247750,15,VT80178720,"Virtus Pharmaceuticals, LLC",Released
3,Oct 3 2022 12:29PM,247749,15,8593042,"Mizner Bioscience, LLC",Released
4,Oct 3 2022 12:26PM,247747,19,8604922,"Innogenix, LLC",Released
5,Oct 3 2022 12:26PM,247747,19,8604923,"Innogenix, LLC",Released
6,Oct 3 2022 12:26PM,247747,19,8604924,"Innogenix, LLC",Released
7,Oct 3 2022 12:26PM,247747,19,8604926,"Innogenix, LLC",Released
8,Oct 3 2022 12:26PM,247747,19,8604927,"Innogenix, LLC",Released
9,Oct 3 2022 12:26PM,247747,19,8604928,"Innogenix, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,247746,Released,6
21,247747,Released,26
22,247749,Released,1
23,247750,Released,1
24,247755,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
247746,NaN,6.0
247747,NaN,26.0
247749,NaN,1.0
247750,NaN,1.0
247755,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
247684,0.0,15.0
247690,0.0,1.0
247692,0.0,1.0
247706,0.0,1.0
247707,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
247684,0,15
247690,0,1
247692,0,1
247706,0,1
247707,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,247684,0,15
1,247690,0,1
2,247692,0,1
3,247706,0,1
4,247707,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,247684,,15
1,247690,,1
2,247692,,1
3,247706,,1
4,247707,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 3 2022 12:55PM,247755,19,ACG North America LLC
1,Oct 3 2022 12:30PM,247719,20,Alumier Labs Inc.
2,Oct 3 2022 12:29PM,247750,15,"Virtus Pharmaceuticals, LLC"
3,Oct 3 2022 12:29PM,247749,15,"Mizner Bioscience, LLC"
4,Oct 3 2022 12:26PM,247747,19,"Innogenix, LLC"
30,Oct 3 2022 12:19PM,247744,15,"Person & Covey, Inc."
47,Oct 3 2022 12:17PM,247746,15,"Alliance Pharma, Inc."
53,Oct 3 2022 12:07PM,247745,19,"AdvaGen Pharma, Ltd"
77,Oct 3 2022 11:54AM,247739,10,"CLINUVEL, Inc."
82,Oct 3 2022 11:49AM,247736,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 3 2022 12:55PM,247755,19,ACG North America LLC,,1
1,Oct 3 2022 12:30PM,247719,20,Alumier Labs Inc.,,1
2,Oct 3 2022 12:29PM,247750,15,"Virtus Pharmaceuticals, LLC",,1
3,Oct 3 2022 12:29PM,247749,15,"Mizner Bioscience, LLC",,1
4,Oct 3 2022 12:26PM,247747,19,"Innogenix, LLC",,26
5,Oct 3 2022 12:19PM,247744,15,"Person & Covey, Inc.",,17
6,Oct 3 2022 12:17PM,247746,15,"Alliance Pharma, Inc.",,6
7,Oct 3 2022 12:07PM,247745,19,"AdvaGen Pharma, Ltd",5,19
8,Oct 3 2022 11:54AM,247739,10,"CLINUVEL, Inc.",,5
9,Oct 3 2022 11:49AM,247736,10,Emerginnova,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 3 2022 12:55PM,247755,19,ACG North America LLC,1,
1,Oct 3 2022 12:30PM,247719,20,Alumier Labs Inc.,1,
2,Oct 3 2022 12:29PM,247750,15,"Virtus Pharmaceuticals, LLC",1,
3,Oct 3 2022 12:29PM,247749,15,"Mizner Bioscience, LLC",1,
4,Oct 3 2022 12:26PM,247747,19,"Innogenix, LLC",26,
5,Oct 3 2022 12:19PM,247744,15,"Person & Covey, Inc.",17,
6,Oct 3 2022 12:17PM,247746,15,"Alliance Pharma, Inc.",6,
7,Oct 3 2022 12:07PM,247745,19,"AdvaGen Pharma, Ltd",19,5
8,Oct 3 2022 11:54AM,247739,10,"CLINUVEL, Inc.",5,
9,Oct 3 2022 11:49AM,247736,10,Emerginnova,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 3 2022 12:55PM,247755,19,ACG North America LLC,1,
1,Oct 3 2022 12:30PM,247719,20,Alumier Labs Inc.,1,
2,Oct 3 2022 12:29PM,247750,15,"Virtus Pharmaceuticals, LLC",1,
3,Oct 3 2022 12:29PM,247749,15,"Mizner Bioscience, LLC",1,
4,Oct 3 2022 12:26PM,247747,19,"Innogenix, LLC",26,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 3 2022 12:55PM,247755,19,ACG North America LLC,1,NaN
1,Oct 3 2022 12:30PM,247719,20,Alumier Labs Inc.,1,NaN
2,Oct 3 2022 12:29PM,247750,15,"Virtus Pharmaceuticals, LLC",1,NaN
3,Oct 3 2022 12:29PM,247749,15,"Mizner Bioscience, LLC",1,NaN
4,Oct 3 2022 12:26PM,247747,19,"Innogenix, LLC",26,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 3 2022 12:55PM,247755,19,ACG North America LLC,1,0.0
1,Oct 3 2022 12:30PM,247719,20,Alumier Labs Inc.,1,0.0
2,Oct 3 2022 12:29PM,247750,15,"Virtus Pharmaceuticals, LLC",1,0.0
3,Oct 3 2022 12:29PM,247749,15,"Mizner Bioscience, LLC",1,0.0
4,Oct 3 2022 12:26PM,247747,19,"Innogenix, LLC",26,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2724862,103,13.0
15,990989,25,0.0
16,743135,11,0.0
19,743247,46,5.0
20,247719,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2724862,103,13.0
1,15,990989,25,0.0
2,16,743135,11,0.0
3,19,743247,46,5.0
4,20,247719,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,103,13.0
1,15,25,0.0
2,16,11,0.0
3,19,46,5.0
4,20,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,103.0
1,15,Released,25.0
2,16,Released,11.0
3,19,Released,46.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Executing,13.0,0.0,0.0,5.0,0.0
Released,103.0,25.0,11.0,46.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Executing,13.0,0.0,0.0,5.0,0.0
1,Released,103.0,25.0,11.0,46.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Executing,13.0,0.0,0.0,5.0,0.0
1,Released,103.0,25.0,11.0,46.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()